# Найти все видео с определенного канала

Воспользовавшись методом search обеспечить поиск и сбор видео с определенных каналов

In [1]:
import pandas as pd
# 1
import yaml
import pathlib
ROOT = pathlib.Path().resolve().parent.parent

with open(ROOT / "config.yml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)


In [2]:
DATA_PATH = ROOT / config['api']['DATA_PATH']

In [3]:
#3
# Variables from config
API_KEY = config['api']['keys']
API_VERSION = config['api']['api_version']
API_SERVICE_NAME = config['api']['api_service_name']

publishedAfter = config['api']['parameters']['date_after']
publishedBefore = config['api']['parameters']['date_before']



In [4]:
# 3.1
class KeyStorage:
    """
    Class to use in iteration
    """

    def __init__(self, key_arr: list[str]):
        self.key_gen = iter(key_arr)

    def get_next_key(self):
        """
        Returns API key to use
        :return:
        """
        return next(self.key_gen)

key_storage = KeyStorage(config['api']['keys'])

In [22]:
# 3.2 Просмотр API ключа
API_KEY = key_storage.get_next_key()

In [23]:
# 4 импортируем и создаем клиент API Youtube
import googleapiclient.discovery as api

youtube = api.build(API_SERVICE_NAME,
                    API_VERSION,
                    developerKey=API_KEY)

## Выделение целевого канала для поиска видео по нему

In [7]:
# 5 Загрузка существующей базы каналов
import pandas as pd

channel_df = pd.read_excel(DATA_PATH / 'channels' / 'channels_union_base.xlsx')

channel_df

,kind,etag,id,snippet.title,snippet.description,snippet.customUrl,snippet.publishedAt,snippet.thumbnails.default.url,snippet.thumbnails.default.width,snippet.thumbnails.default.height,...,topicDetails.topicIds,topicDetails.topicCategories,brandingSettings.image.bannerExternalUrl,snippet.country,brandingSettings.channel.unsubscribedTrailer,brandingSettings.channel.country,brandingSettings.channel.keywords,snippet.defaultLanguage,brandingSettings.channel.defaultLanguage,brandingSettings.channel.trackingAnalyticsAccountId
0,youtube#channel,AbINH4P8C3KuSGZNXMKcuJNQiUw,UCvGowJMOfZJ8ozZmOTg5EZQ,AVOCADOKOREAN Корейская косметика,Привет! \nРады видеть тебя на нашем канале AVO...,@avocadokorean6502,2015-11-04T16:13:22Z,https://yt3.ggpht.com/3uaGBOEEMEiKZHwA_akaHRJs...,88,88,...,['/m/019_rr'],['https://en.wikipedia.org/wiki/Lifestyle_(soc...,https://yt3.googleusercontent.com/rsumaOZ4k8kS...,RU,lR_qufVQCB0,RU,"""корейская косметика"" косметика санкт-петербург",NaN,NaN,NaN
1,youtube#channel,PgUoocZZ4stQ_yK7FKJHuv4JzCY,UCSoadOaxY49VhoAvchaw5eg,Mark Kaufman,INSTAGRAM @KAUFMANMK,@kaufmanlifestyle,2014-01-24T16:19:56Z,https://yt3.ggpht.com/ytc/AL5GRJVSRHeb4AIhdqya...,88,88,...,['/m/019_rr'],['https://en.wikipedia.org/wiki/Lifestyle_(soc...,https://lh3.googleusercontent.com/d-t_MV0NLYCH...,RU,eCM-sNaHPAo,RU,NaN,NaN,NaN,NaN
2,youtube#channel,0ScIKieZR7jwO1NLcgyqPh97Nho,UCXNXpk6BPZO-M-fYyvfF06Q,Bo Hyun Kim,Здесь я делюсь с Вами полезной информацией о к...,@bohyunkim8050,2016-08-08T18:44:38Z,https://yt3.ggpht.com/ytc/AL5GRJVedf_bXWhv_yh8...,88,88,...,['/m/019_rr'],['https://en.wikipedia.org/wiki/Lifestyle_(soc...,https://yt3.googleusercontent.com/j4gHcr7pjMA9...,NaN,oIDxY1nZfAs,NaN,"корея ""корейская культура"" ""корейская косметик...",NaN,NaN,NaN
3,youtube#channel,UdR-B20fBqnZ8YZ6Cg_acTOtkiM,UCIyBPZOkOrXIsRPhhdunEiQ,bb-mania.kz - лучший интернет-магазин Корейско...,Корейская косметика от известных брендов:\n*Mi...,@bbmaniakz,2014-07-06T16:25:52Z,https://yt3.ggpht.com/ytc/AL5GRJXPTP9me76i5FJb...,88,88,...,"['/m/019_rr', '/m/041xxh']",['https://en.wikipedia.org/wiki/Lifestyle_(soc...,https://yt3.googleusercontent.com/n8KPZef6z5Cf...,KZ,XhwKaBR2lXM,KZ,"""лучшие корейские интернет магазины"" ""корейска...",NaN,NaN,NaN
4,youtube#channel,LIec9POKYATeYNY-CyRm4rHhQCQ,UCohziQCjyBd7h1xFakUpXzQ,LuckyCosmetics Корейская косметика,Инстаграм @luckycosmetics.ru\n\nВсем привет! С...,@luckycosmetics_shop,2018-07-13T15:24:25Z,https://yt3.ggpht.com/ytc/AL5GRJXu8OyMkOVzMkiy...,88,88,...,"['/m/019_rr', '/m/041xxh']",['https://en.wikipedia.org/wiki/Lifestyle_(soc...,https://lh3.googleusercontent.com/pWG4BBdA4jgC...,RU,HNMJ4wpEeiU,RU,"""Корейская косметика"" ""корейская косметика отз...",NaN,NaN,UA-15256516-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,youtube#channel,2Uv7Lpv-P_xlz4LmGN_O41mGAA4,UCCz7PAUdGLfoXYgapfe8JSA,Секреты юной леди,"Привет, подружка! Если у тебя есть вопросы, о ...",@sekrety_ledi,2019-09-19T10:44:26Z,https://yt3.ggpht.com/EDNjbGT1_xRQnx1HXXe5JkQY...,88,88,...,['/m/019_rr'],['https://en.wikipedia.org/wiki/Lifestyle_(soc...,https://yt3.googleusercontent.com/SuMnDc3AAHlW...,UA,zG_y7rXiOL4,UA,"""секреты юной леди"" ""советы юной леди"" секреты...",ru,ru,NaN
201,youtube#channel,rfyJbO4Dh9oZxtmQm80wmCCmxsA,UCHX4WAfPLgGwqq35KaqGy0A,Бабушка Шошо,Всем привет!!!\nМеня зовут бабушка Шошо. \nНа ...,@cartoonshosho,2012-10-11T19:30:40Z,https://yt3.ggpht.com/ytc/AL5GRJWt8By4ipmlWzKt...,88,88,...,"['/m/02vxn', '/m/01k8wb', '/m/02jjt']","['https://en.wikipedia.org/wiki/Film', 'https:...",https://lh3.googleusercontent.com/PgIzqRQHqcp3...,RU,mJ0cPfnrey8,RU,"""тесты на логику"" ""загадки с ответами"" ""тесты ...",NaN,NaN,NaN
202,youtube#channel,PALuvDlrxG0Ke_UNJx6uONL1S9o,UCFKyV7bEwQcEeOhgBFP_PoA,Точка Любви,Добро пожаловать на канал сети магазинов для в...,@tochka_lubvi,2018-10-02T14:32:47Z,https://yt3.ggpht.com/Qb7LudPEoZdj98BSdtouNUW2...,88,88,...,"['/m/019_rr', '/m/0kt51', '/m/03glg']",['https://en.wikipedia.org/w

In [8]:
# 6 просмотр информации о DataFrame с каналами

channel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 36 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   kind                                                 205 non-null    object 
 1   etag                                                 205 non-null    object 
 2   id                                                   205 non-null    object 
 3   snippet.title                                        205 non-null    object 
 4   snippet.description                                  194 non-null    object 
 5   snippet.customUrl                                    205 non-null    object 
 6   snippet.publishedAt                                  205 non-null    object 
 7   snippet.thumbnails.default.url                       205 non-null    object 
 8   snippet.thumbnails.default.width                     205 non-null    i

In [9]:
QUERY = 'redautumn'

In [10]:
# 7 Сбор id каналов посредством поиска

def get_id_by_word(df: pd.DataFrame, query: str) -> list[str]:

    ids = df.loc[df['snippet.title'].str.contains(query, case=False, regex=True), 'id']
    return list(ids)


targets = get_id_by_word(channel_df, QUERY)
targets

['UCaEh6lSHDzZXju9cfYrmI0g']

## Сбор видео с определенного канала

In [12]:
# 8
CONTENT_TYPE = 'video'
TARGET_ID = targets[0]


In [14]:
# 9 Поиск видео с определенного канал

# 18.1 Итоговый собранный код из прошлых видео без промежуточных шагов
# Первый заход БЕЗ аргумента order:
request = youtube.search().list(
    part="snippet"
    , maxResults=50
    , type=CONTENT_TYPE
    , channelId = TARGET_ID
)
response = request.execute()

# Занести собранные данные в итоговую таблицу df_supplemented:
df_supplemented = pd.json_normalize(response['items'])

# Цикл для прохода по всем следующим страницам с выдачей:
i = 0
while 'nextPageToken' in response.keys():
    request = youtube.search().list(
        part="snippet"
        , maxResults=50
        , type=CONTENT_TYPE
        , channelId = TARGET_ID
        , pageToken=response['nextPageToken']
    )
    response = request.execute()

    # Для визуализации процесса:
    print(f'Итерация БЕЗ включения аргумента order №{i}')
    i += 1

    # Занести собранные данные в итоговую таблицу df_supplemented:
    df_additional = pd.json_normalize(response['items'])
    df_supplemented = pd.concat([df_supplemented, df_additional])

print(f"Искомых объектов {response['pageInfo']['totalResults']}, \
      а найденных БЕЗ включения каких-либо значений аргумента order {len(df_supplemented.drop_duplicates('snippet.description'))}")

Итерация БЕЗ включения аргумента order №0
Итерация БЕЗ включения аргумента order №1
Итерация БЕЗ включения аргумента order №2
Итерация БЕЗ включения аргумента order №3
Итерация БЕЗ включения аргумента order №4
Итерация БЕЗ включения аргумента order №5
Итерация БЕЗ включения аргумента order №6
Итерация БЕЗ включения аргумента order №7
Итерация БЕЗ включения аргумента order №8
Итерация БЕЗ включения аргумента order №9
Искомых объектов 1589,       а найденных БЕЗ включения каких-либо значений аргумента order 396


In [15]:
# 10 Значения аргумента order:
order_list = ['date', 'rating', 'title', 'videoCount', 'viewCount']
order_list

['date', 'rating', 'title', 'videoCount', 'viewCount']

In [24]:
# 11 Ротация аргументов ордер для сбора оставшихся видео с каналов

# 18.2 Итоговый собранный код из прошлых видео без промежуточных шагов
# Цикл для прохода по значениям аргумента order, внутри которых проход по всем страницам выдачи:
# Для остановки алгоритма, если все искомые объекты найдены
# БЕЗ включения каких-либо значений аргумента order (в т.ч. вообще БЕЗ них):
if len(df_supplemented.drop_duplicates('snippet.description')) < response['pageInfo']['totalResults']: # !!!
    i = 0
    for order in order_list:
        # Первый заход с каждым значением аргумента order:
        request = youtube.search().list(
            part="snippet"
            , maxResults=50
        #     ,q=q
            , type=CONTENT_TYPE
            , channelId = TARGET_ID
            , order=order
        )
        response = request.execute()

        # Для визуализации процесса:
        print(f'Итерация №{i}, "order" {order}, "items" {len(response["items"])}')
        i += 1

        # Занести собранные данные в итоговую таблицу df_supplemented:
        df_additional = pd.json_normalize(response['items'])
        df_supplemented = pd.concat([df_additional, df_supplemented])

        # Заходы с тем же значением аргумента order на следующие страницы:
        while ('nextPageToken' in response.keys())\
        & (len(df_supplemented.drop_duplicates('snippet.description')) < response['pageInfo']['totalResults'])\
        & (len(response["items"]) > 0): # !!!
        # второе условие -- для остановки алгоритма, если все искомые объекты найдены
        # БЕЗ какой-то из следующих страниц (в т.ч. вообще БЕЗ них)
        # третье условие -- для остановки алгоритма, если предыдущая страница выдачи содержит 0 объектов

            request = youtube.search().list(
                part="snippet"
                , maxResults=50
            #     ,q=q
                , type=CONTENT_TYPE
                , channelId = TARGET_ID
                , order=order
                , pageToken=response['nextPageToken']
            )
            response = request.execute()

            # Для визуализации процесса:
            print(f'Итерация №{i}, "order" {order}, "items" {len(response["items"])}')
            i+=1

            # Занести собранные данные в итоговую таблицу df_supplemented:
            df_additional = pd.json_normalize(response['items'])
            df_supplemented = pd.concat([df_supplemented, df_additional])

else:
    print('Все искомые объекты найдены БЕЗ включения некоторых значений аргумента order (в т.ч. вообще БЕЗ них)')

Итерация №0, "order" date, "items" 50
Итерация №1, "order" date, "items" 50
Итерация №2, "order" date, "items" 50
Итерация №3, "order" date, "items" 50
Итерация №4, "order" date, "items" 50
Итерация №5, "order" date, "items" 50
Итерация №6, "order" date, "items" 50
Итерация №7, "order" date, "items" 50
Итерация №8, "order" date, "items" 50
Итерация №9, "order" date, "items" 50
Итерация №10, "order" date, "items" 2
Итерация №11, "order" rating, "items" 50
Итерация №12, "order" rating, "items" 50
Итерация №13, "order" rating, "items" 50
Итерация №14, "order" rating, "items" 50
Итерация №15, "order" rating, "items" 50
Итерация №16, "order" rating, "items" 50
Итерация №17, "order" rating, "items" 50
Итерация №18, "order" rating, "items" 50
Итерация №19, "order" rating, "items" 50
Итерация №20, "order" rating, "items" 50
Итерация №21, "order" rating, "items" 3
Итерация №22, "order" title, "items" 50
Итерация №23, "order" title, "items" 50
Итерация №24, "order" title, "items" 50
Итерация №25

In [25]:
# 12
df_supplemented = df_supplemented.drop_duplicates('snippet.description')
df_supplemented

,kind,etag,id.kind,id.videoId,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,snippet.thumbnails.default.height,snippet.thumbnails.medium.url,snippet.thumbnails.medium.width,snippet.thumbnails.medium.height,snippet.thumbnails.high.url,snippet.thumbnails.high.width,snippet.thumbnails.high.height,snippet.channelTitle,snippet.liveBroadcastContent,snippet.publishTime
0,youtube#searchResult,5qk2AqqnhQHhtnSIjszH9fHp5EM,youtube#video,DWERqf7DdqQ,2017-02-02T05:30:00Z,UCaEh6lSHDzZXju9cfYrmI0g,"МАКИЯЖ ДЛЯ НАЧИНАЮЩИХ: ЧТО, ЗАЧЕМ и КАК?",Новое видео! МАКИЯЖ ДЛЯ НАЧИНАЮЩИХ: БЮДЖЕТНАЯ ...,https://i.ytimg.com/vi/DWERqf7DdqQ/default.jpg,120,90,https://i.ytimg.com/vi/DWERqf7DdqQ/mqdefault.jpg,320,180,https://i.ytimg.com/vi/DWERqf7DdqQ/hqdefault.jpg,480,360,RedAutumn,none,2017-02-02T05:30:00Z
1,youtube#searchResult,1tVBZ9KMS7h2GvwkDJcCWnWjbus,youtube#video,BHD_SB04izE,2021-11-14T07:00:04Z,UCaEh6lSHDzZXju9cfYrmI0g,ЭТО ПОЗОР ЛЮКСА!!! (АД🤬)ВЕНТ КАЛЕНДАРЬ CHANEL ...,адвент_календарь #chanel #адвент_календарь_шан...,https://i.ytimg.com/vi/BHD_SB04izE/default.jpg,120,90,https://i.ytimg.com/vi/BHD_SB04izE/mqdefault.jpg,320,180,https://i.ytimg.com/vi/BHD_SB04izE/hqdefault.jpg,480,360,RedAutumn,none,2021-11-14T07:00:04Z
2,youtube#searchResult,OoxzUDOTArUdRIfHSYRt9c4jttY,youtube#video,6G-pOUjeV8E,2021-07-12T04:00:00Z,UCaEh6lSHDzZXju9cfYrmI0g,Работает ли ВОЛШЕБНАЯ ЗЕЛЕНАЯ МАСКА из ТИК ТОК?,тик_ток #зеленая_маска #green_mask #зеленая_ма...,https://i.ytimg.com/vi/6G-pOUjeV8E/default.jpg,120,90,https://i.ytimg.com/vi/6G-pOUjeV8E/mqdefault.jpg,320,180,https://i.ytimg.com/vi/6G-pOUjeV8E/hqdefault.jpg,480,360,RedAutumn,none,2021-07-12T04:00:00Z
3,youtube#searchResult,H03C-aiimmJTMVKfMLCHYwtlG4g,youtube#video,wTf3Nu3Q5sA,2021-11-06T07:00:30Z,UCaEh6lSHDzZXju9cfYrmI0g,НЕРЕАЛЬНЫЕ АДВЕНТ КАЛЕНДАРИ DIOR!!! | DIOR ADV...,адвент_календарь #dior #адвент_календарь_диор ...,https://i.ytimg.com/vi/wTf3Nu3Q5sA/default.jpg,120,90,https://i.ytimg.com/vi/wTf3Nu3Q5sA/mqdefault.jpg,320,180,https://i.ytimg.com/vi/wTf3Nu3Q5sA/hqdefault.jpg,480,360,RedAutumn,none,2021-11-06T07:00:30Z
4,youtube#searchResult,MLWUaTo3mFZyqXDgAAFUIQlUjf8,youtube#video,cCfnUOkSUB4,2019-12-22T04:00:06Z,UCaEh6lSHDzZXju9cfYrmI0g,МЫ ДОЖДАЛИСЬ! 😱MAC - ЛУЧШИЙ АДВЕНТ КАЛЕНДАРЬ? ...,адвент #адвент_календарь #redautumn #адвент_ма...,https://i.ytimg.com/vi/cCfnUOkSUB4/default.jpg,120,90,https://i.ytimg.com/vi/cCfnUOkSUB4/mqdefault.jpg,320,180,https://i.ytimg.com/vi/cCfnUOkSUB4/hqdefault.jpg,480,360,RedAutumn,none,2019-12-22T04:00:06Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48,youtube#searchResult,sPpGwUomQt3ydyr8JjDLnoy8-P4,youtube#video,YhkNzKcgEJ8,2018-12-29T04:00:02Z,UCaEh6lSHDzZXju9cfYrmI0g,ТАК ЧТО ДАРИТЬ-ТО?! 😱 ГИД ПО ПОДАРКАМ 2019 В С...,В этом видео я расскажу вам о гиде по новогодн...,https://i.ytimg.com/vi/YhkNzKcgEJ8/default.jpg,120,90,https://i.ytimg.com/vi/YhkNzKcgEJ8/mqdefault.jpg,320,180,https://i.ytimg.com/vi/YhkNzKcgEJ8/hqdefault.jpg,480,360,RedAutumn,none,2018-12-29T04:00:02Z
14,youtube#searchResult,ozeXi7su0Iw3Wucpg7Zw23vQJoE,youtube#video,HbNWNEbm0Wg,2018-11-29T04:00:02Z,UCaEh6lSHDzZXju9cfYrmI0g,"КАК КРАСИТЬСЯ, ЧТОБЫ НЕ БЫЛО СТЫДНО? МАКИЯЖ ДЛ...",Скидка на пудры L'Oreal на Wildberries - https...,https://i.ytimg.com/vi/HbNWNEbm0Wg/default.jpg,120,90,https://i.ytimg.com/vi/HbNWNEbm0Wg/mqdefault.jpg,320,180,https://i.ytimg.com/vi/HbNWNEbm0Wg/hqdefault.jpg,480,360,RedAutumn,none,2018-11-29T04:00:02Z
25,youtube#searchResult,C8VcoR70KagQ1MMguIniD-5MO5s,youtube#video,wwqaYpv4Keg,2018-10-19T04:00:03Z,UCaEh6lSHDzZXju9cfYrmI0g,ГОРЯЧО! 🔥БЮДЖЕТНЫЕ НОВИНКИ CATRICE,В этом видео я покажу вам бюджетные новинки от...,https://i.ytimg.com/vi/wwqaYpv4Keg/default.jpg,120,90,https://i.ytimg.com/vi/wwqaYpv4Keg/mqdefault.jpg,320,180,https://i.ytimg.com/vi/wwqaYpv4Keg/hqdefault.jpg,480,360,RedAutumn,none,2018-10-19T04:00:03Z
24,youtube#searchResult,zEAxwhNB9YeKrt8e0v0kJsSqPas,youtube#video

In [26]:
# 13 Взять из таблицы с найденными ранее видео их id (как в предыдущем чанке)
videoId_list = df_supplemented[f'id.{CONTENT_TYPE}Id'].to_list()
videoId_list

['DWERqf7DdqQ',
 'BHD_SB04izE',
 '6G-pOUjeV8E',
 'wTf3Nu3Q5sA',
 'cCfnUOkSUB4',
 'DCk5YkaX8uU',
 'bXD518tCy0k',
 'qqItDCDLDbk',
 'kvBI84RLhZU',
 'Hrh57HILRIQ',
 'QbACOmOaRUw',
 'TE3PY43WcRg',
 'wY5oNjVN5tI',
 '0PeLvdFrb6c',
 'tIJRcnqlhW8',
 'j6U67ZdSStw',
 'k-2tk-HBhqU',
 'Ut4bs712gsI',
 'LluMqTfw_i0',
 'LwUl3FR3bZM',
 'bZPIk78nx-8',
 'bGWke49V1ZI',
 'T-etDi9Qlb0',
 'efkf_C41mOc',
 'RKi-c8BYGsw',
 'VGJlQ1FJje4',
 '0Y7kwCA47i0',
 'Okq-6FEdPj0',
 '2u78UJ4VvYY',
 'jhaBRS5V9Kc',
 'Xn_cl9bhmqg',
 'Fl4Ah6oxltQ',
 'Ytew78-7q5o',
 'R-Di5YpHvW4',
 '0okpTBNvnzk',
 '2O48egkHoUE',
 '2PNuRDhNt8w',
 'FSyXRWmpvBE',
 'CJ6ynCJrWOo',
 'O-ALzYrdluo',
 'Kpx0CHuhenw',
 'bOWsp5V-KJs',
 '9RpNNJht7i0',
 'sqwW3CR99s0',
 '7eTNmtQj6VU',
 '8qxsIlg8s08',
 'JigCnhI1CjE',
 'ZdPWYo5hm3E',
 '-LO4zybhqZw',
 'AuughuJKvdI',
 'K_ND2ICXCSM',
 'PZ_81aZViPk',
 'xG3C3SFIjO8',
 'tmDwp-qOTM0',
 'StS_ZmsDHIQ',
 'KDgEi79NQBU',
 'q7vcBu-ih5c',
 'LqEffqSmNEI',
 'lwBP-s9Bj5c',
 'T4N-cMfZHGs',
 'WpjfX8mrgZ8',
 '7mwQeCj8z5I',
 'cGdwAb

In [27]:
# 14
print(f'До удаления дубликатов id в списке было {len(videoId_list)} элементов,',\
      f'после удаления -- {len(list(dict.fromkeys(videoId_list)))} элементов',
      sep='\n')
videoId_list = list(dict.fromkeys(videoId_list))

До удаления дубликатов id в списке было 427 элементов,
после удаления -- 427 элементов


In [28]:
# 15 Цикл для прохода по всем порциям по 50 объектов
df_supplemented = pd.DataFrame()
for i in range(0, len(videoId_list), 50):
    request = youtube.videos().list(
        part=["snippet",
              "contentDetails",
              "localizations",
              "statistics",
              "status",
              "topicDetails"],
        id=videoId_list[i:i + 50],
        maxResults=50
    )
    response = request.execute()

    print(f'Итерация №{i}')

    df_additional = pd.json_normalize(response['items'])
    df_supplemented = pd.concat([df_supplemented, df_additional]) # соединить две таблицы

Итерация №0
Итерация №50
Итерация №100
Итерация №150
Итерация №200
Итерация №250
Итерация №300
Итерация №350
Итерация №400


In [29]:
# 16 Удалить дубликаты видео (по их id) и записать в Excel:
df_supplemented = df_supplemented.drop_duplicates('id')
df_supplemented.index = range(1, len(df_supplemented) + 1)
display(df_supplemented)


,kind,etag,id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,snippet.thumbnails.default.height,...,status.license,status.embeddable,status.publicStatsViewable,status.madeForKids,statistics.viewCount,statistics.likeCount,statistics.favoriteCount,statistics.commentCount,topicDetails.topicCategories,contentDetails.regionRestriction.blocked
1,youtube#video,tLftXEDLmRbEvz2SzulwaRRdycs,DWERqf7DdqQ,2017-02-02T05:30:00Z,UCaEh6lSHDzZXju9cfYrmI0g,"МАКИЯЖ ДЛЯ НАЧИНАЮЩИХ: ЧТО, ЗАЧЕМ и КАК?",Новое видео! МАКИЯЖ ДЛЯ НАЧИНАЮЩИХ: БЮДЖЕТНАЯ ...,https://i.ytimg.com/vi/DWERqf7DdqQ/default.jpg,120,90,...,youtube,True,True,False,3721739,89228,0,2015,[https://en.wikipedia.org/wiki/Lifestyle_(soci...,NaN
2,youtube#video,ZxpoBb8SZGTRI7PbhQ_p-2JLpOY,BHD_SB04izE,2021-11-14T07:00:04Z,UCaEh6lSHDzZXju9cfYrmI0g,ЭТО ПОЗОР ЛЮКСА!!! (АД🤬)ВЕНТ КАЛЕНДАРЬ CHANEL ...,#адвент_календарь #chanel #адвент_календарь_ша...,https://i.ytimg.com/vi/BHD_SB04izE/default.jpg,120,90,...,youtube,True,True,False,2344271,118385,0,14761,[https://en.wikipedia.org/wiki/Lifestyle_(soci...,NaN
3,youtube#video,Me4oVZYgI0Gmv4bGXUYScze_AeU,6G-pOUjeV8E,2021-07-12T04:00:00Z,UCaEh6lSHDzZXju9cfYrmI0g,Работает ли ВОЛШЕБНАЯ ЗЕЛЕНАЯ МАСКА из ТИК ТОК?,#тик_ток #зеленая_маска #green_mask #зеленая_м...,https://i.ytimg.com/vi/6G-pOUjeV8E/default.jpg,120,90,...,youtube,True,True,False,1331241,32005,0,898,[https://en.wikipedia.org/wiki/Lifestyle_(soci...,NaN
4,youtube#video,GTOI01rZHpi_tjNpURuitEnxTBQ,wTf3Nu3Q5sA,2021-11-06T07:00:30Z,UCaEh6lSHDzZXju9cfYrmI0g,НЕРЕАЛЬНЫЕ АДВЕНТ КАЛЕНДАРИ DIOR!!! | DIOR ADV...,#адвент_календарь #dior #адвент_календарь_диор...,https://i.ytimg.com/vi/wTf3Nu3Q5sA/default.jpg,120,90,...,youtube,True,True,False,911996,39333,0,1659,[https://en.wikipedia.org/wiki/Lifestyle_(soci...,NaN
5,youtube#video,Qnmow1YaXHw62JBCeaWvI2bdTcU,cCfnUOkSUB4,2019-12-22T04:00:06Z,UCaEh6lSHDzZXju9cfYrmI0g,МЫ ДОЖДАЛИСЬ! 😱MAC - ЛУЧШИЙ АДВЕНТ КАЛЕНДАРЬ? ...,#адвент #адвент_календарь #redautumn #адвент_м...,https://i.ytimg.com/vi/cCfnUOkSUB4/default.jpg,120,90,...,youtube,True,True,False,904637,35289,0,807,[https://en.wikipedia.org/wiki/Lifestyle_(soci...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,youtube#video,6hglPsl-DogbL8cg_mbeTR8wGV0,YhkNzKcgEJ8,2018-12-29T04:00:02Z,UCaEh6lSHDzZXju9cfYrmI0g,ТАК ЧТО ДАРИТЬ-ТО?! 😱 ГИД ПО ПОДАРКАМ 2019 В С...,В этом видео я расскажу вам о гиде по новогодн...,https://i.ytimg.com/vi/YhkNzKcgEJ8/default.jpg,120,90,...,youtube,True,True,False,73723,3204,0,149,[https://en.wikipedia.org/wiki/Lifestyle_(soci...,NaN
424,youtube#video,QlM2oG-R1201pZ7Fi7E0e_DA1zU,HbNWNEbm0Wg,2018-11-29T04:00:02Z,UCaEh6lSHDzZXju9cfYrmI0g,"КАК КРАСИТЬСЯ, ЧТОБЫ НЕ БЫЛО СТЫДНО? МАКИЯЖ ДЛ...",Скидка на пудры L’Oreal на Wildberries - https...,https://i.ytimg.com/vi/HbNWNEbm0Wg/default.jpg,120,90,...,youtube,True,True,False,54444,2449,0,64,[https://en.wikipedia.org/wiki/Lifestyle_(soci...,NaN
425,youtube#video,CUFWq1kyUjxbLgizB92nc7IwW-c,wwqaYpv4Keg,2018-10-19T04:00:03Z,UCaEh6lSHDzZXju9cfYrmI0g,ГОРЯЧО! 🔥БЮДЖЕТНЫЕ НОВИНКИ CATRICE,В этом видео я покажу вам бюджетные новинки от...,https://i.ytimg.com/vi/wwqaYpv4Keg/default.jpg,120,90,...,youtube,True,True,False,76703,2734,0,86,[https://en.wikipedia.org/wiki/Lifestyle_(soci...,NaN
426,youtube#video,eWzyXIaUb51GuAPAZhGUTVH-qyE,3-BAcFN0cmU,2019-03-22T04:00:00Z,UCaEh6lSHDzZXju9cfYrmI0g,"АНТИПОКУПКИ: КРУТЫЕ НОВИНКИ КОСМЕТИКИ, КОТОРЫЕ...","В этом видео я расскажу вам о той косметике, к...",https://i.ytimg.com/vi/3-BAcFN0cmU/default.jpg,120,90,...,youtube,True,True,False,122365,4759,0,333,[https://en.wikipedia.org/wiki/Lifestyle_(soci...,NaN


In [30]:
#17
import os

SAVE_PATH = DATA_PATH / 'channels_video'

if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)

df_supplemented.to_excel(SAVE_PATH /  f'{CONTENT_TYPE}_{QUERY}.xlsx', index=False)